# Training
Seems like we're ready for training!

Be aware of:
IP : 131.114.137.168

username : boscolo

pssw : unipi!2020$



stai attento a usare al max 2 GPU

In [1]:
import sys, os, datetime
sys.path.append('/home/andrea/AI/ispr_yolo/data')
sys.path.append('/home/andrea/AI/ispr_yolo/model')
from yolo import YOLOv3
from lossfunction import YoloLoss
from DataPreprocessing import create_dataset
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard

In [1]:
!rm -rf logs
!mkdir logs logs/fit logs/scalars

In [3]:
%load_ext tensorboard
!kill 813
%tensorboard --logdir logs --port 6001 --host=0.0.0.0

/bin/sh: 1: kill: No such process



Reusing TensorBoard on port 6001 (pid 813), started 2 days, 0:32:27 ago. (Use '!kill 813' to kill it.)

In [4]:
def scheduler(epoch, lr):
    epoch=epoch+1
    if epoch < 4:
        print('Learning rate for epoch n.', epoch, ' :', lr)
        return lr
    elif epoch < 8:
        print('Learning rate for epoch n.', epoch, ' :', lr*0.8)
        return lr * 0.8
    else:
        print('Learning rate for epoch n.', epoch, ' :', lr*0.2)
        return lr * 0.2

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
with tf.device('/cpu:0'):
    img_dir = '/home/andrea/AI/ispr_yolo/data/dataset_bdd/images/100k/train' + '/*.jpg'
    train_ds_aug = create_dataset(img_dir, batch = 24, augmented = True)
    img_dir = '/home/andrea/AI/ispr_yolo/data/dataset_bdd/images/100k/val' + '/*.jpg'
    val_ds = create_dataset(img_dir, batch = 24, validation = True)

strategy = tf.distribute.MirroredStrategy(devices=["/gpu:2", "/gpu:3"])
with strategy.scope():
    log_dir = os.path.join("logs/fit", datetime.datetime.now().strftime("%m%d-%H%M%S"))
    yolo = YOLOv3(size = 1280, training = True)
    nadam = tf.keras.optimizers.Nadam(learning_rate=1e-03) #2e-04
    yolo.compile(loss=[YoloLoss(10, 'dense', softmax = False),
                       YoloLoss(10, 'medium', softmax = False),
                       YoloLoss(10, 'coarse', softmax = False)],
                       optimizer=nadam)


    log_dir = os.path.join("logs/fit", datetime.datetime.now().strftime("%m%d-%H%M%S"))
    logging = TensorBoard(log_dir)
    checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
                                 monitor='val_loss', save_weights_only=True,
                                 save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1)

    yolo.fit(train_ds_aug,
             epochs = 20,
             callbacks = [logging, checkpoint, lr_scheduler, early_stopping],
             validation_data = val_ds)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensor